In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import os

os.chdir("/Users/BharathBandaru/Desktop/Assignments/NLP/A3/A_2_output")
fl = os.listdir()
df_list = []
for i in fl:
    print(i)
    df = pd.read_csv(i)
    df_list.append(df)
    #print(df.describe())
    a = list(df.label)
    print("True Count: {0} False count {1}".format(
        a.count(True), a.count(False)),"\n")

twitter_topic_lockdowns.csv
True Count: 55 False count 1140 

twitter_topic_masking_and_distancing.csv
True Count: 267 False count 928 

change.org_topic_masking_and_distancing.csv
True Count: 62 False count 1438 

change.org_topic_lockdowns.csv
True Count: 439 False count 1061 

change.org_topic_final_label.csv
True Count: 0 False count 0 

twitter_topic_vaccination.csv
True Count: 687 False count 508 

twitter_final_label.csv
True Count: 0 False count 0 

change.org_topic_vaccination.csv
True Count: 26 False count 1474 



In [15]:
import numpy
import pandas as pd

#we choose vaccination
df = pd.read_csv("/Users/BharathBandaru/Desktop/Assignments/NLP/A3/A_2_output/twitter_topic_vaccination.csv")
print('Shape of the data: ', df.shape)
df.head()

Shape of the data:  (1195, 2)


,text,label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [16]:
#text pre-processing imports

import re
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize as wt 

nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

from sklearn.linear_model import LogisticRegression

#spell correction
from autocorrect import Speller

spell = Speller(lang='en')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/BharathBandaru/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/BharathBandaru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
#text pre-processing
data = []

for i in range(df.shape[0]):
    txt = df.iloc[i, 0]
    
    #remove alphabets
    txt = re.sub('[^A-Za-z]', ' ', txt)
    
    txt = txt.lower()
    
    tokenized_txts = wt(txt)
    
    text_processed = []
    for word in tokenized_txts:
        if word not in set(stopwords.words('english')):
            text_processed.append(spell(stemmer.stem(word)))
            
    final_text = " ".join(text_processed)
    data.append(final_text)

In [18]:
sec_df = pd.read_csv("/Users/BharathBandaru/Desktop/Assignments/NLP/A3/A_2_output/change.org_topic_vaccination.csv")

In [19]:
#SECONDARY DATA
sec_data = []

for i in range(sec_df.shape[0]):
    txt = sec_df.iloc[i, 0]
    
    #remove alphabets
    txt = re.sub('[^A-Za-z]', ' ', txt)
    
    txt = txt.lower()
    
    tokenized_txts = wt(txt)
    
    text_processed = []
    for word in tokenized_txts:
        if word not in set(stopwords.words('english')):
            text_processed.append(spell(stemmer.stem(word)))
            
    final_text = " ".join(text_processed)
    sec_data.append(final_text)

In [20]:
# creating the feature matrix 
from sklearn.feature_extraction.text import CountVectorizer

matrix = CountVectorizer(max_features=1000)
X = matrix.fit_transform(data).toarray()
y = df.iloc[:, 1]

In [21]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

X_test_sec = matrix.fit_transform(sec_data).toarray()
y_test_sec = sec_df.iloc[:, 1]

print(len(X_train))
print(len(X_test))

896
299


In [22]:
lr_model = LogisticRegression(penalty="none", 
                              multi_class="multinomial",
                              solver="lbfgs").fit(X_train, y_train)
y_lr_test = lr_model.predict(X_test)
y_lr_sec_test = lr_model.predict(X_test_sec)

In [24]:
#PRIMARY TEST
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(classification_report(y_test, y_lr_test))
print("ACCURACY: ",accuracy_score(y_test, y_lr_test))

              precision    recall  f1-score   support

       False       0.91      0.84      0.87       143
        True       0.86      0.92      0.89       156

    accuracy                           0.88       299
   macro avg       0.89      0.88      0.88       299
weighted avg       0.88      0.88      0.88       299

ACCURACY:  0.882943143812709


In [25]:
#SECONDARY
print(classification_report(y_test_sec, y_lr_sec_test))
print("ACCURACY: ",accuracy_score(y_test_sec, y_lr_sec_test))

              precision    recall  f1-score   support

       False       0.99      0.57      0.72      1474
        True       0.02      0.54      0.04        26

    accuracy                           0.56      1500
   macro avg       0.50      0.55      0.38      1500
weighted avg       0.97      0.56      0.71      1500

ACCURACY:  0.5646666666666667


In [26]:
lr2_model = LogisticRegression(penalty="l2", 
                               solver="lbfgs",
                               multi_class="multinomial",
                               max_iter=1000,
                               C=10).fit(X_train, y_train)
y_lr2_test = lr2_model.predict(X_test)
y_lr2_sec_test = lr2_model.predict(X_test_sec)

In [27]:
#PRIMARY TEST
print(classification_report(y_test, y_lr2_test))
print("ACCURACY: ",accuracy_score(y_test, y_lr2_test))

              precision    recall  f1-score   support

       False       0.91      0.88      0.89       143
        True       0.89      0.92      0.91       156

    accuracy                           0.90       299
   macro avg       0.90      0.90      0.90       299
weighted avg       0.90      0.90      0.90       299

ACCURACY:  0.8996655518394648


In [28]:
#SECONDARY
print(classification_report(y_test_sec, y_lr2_sec_test))
print("ACCURACY: ",accuracy_score(y_test_sec, y_lr2_sec_test))

              precision    recall  f1-score   support

       False       0.98      0.83      0.90      1474
        True       0.02      0.23      0.04        26

    accuracy                           0.82      1500
   macro avg       0.50      0.53      0.47      1500
weighted avg       0.97      0.82      0.89      1500

ACCURACY:  0.8213333333333334


In [29]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [30]:
# predict class
y_pred = classifier.predict(X_test)
sec_y_pred = classifier.predict(X_test_sec)

In [31]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

sec_cm = confusion_matrix(y_test_sec, sec_y_pred)
sec_cr = classification_report(y_test_sec, sec_y_pred)

In [32]:
#accuracy of primary test data
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.6755852842809364


In [33]:
#accuracy of secondary test data
sec_data_accuracy = accuracy_score(y_test_sec, sec_y_pred)
print(sec_data_accuracy)

0.4706666666666667


In [65]:
X_df_train, X_df_test, y_train_2, y_test_2 = train_test_split(df["text"],y)

print(X_df_train.shape)
print(X_df_test.shape)
print(y_train_2.shape)
print(y_test_2.shape)

(896,)
(299,)
(896,)
(299,)


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3, stop_words="english").fit(X_df_train)
X_train_2 = vectorizer.transform(X_df_train)
X_test_2 = vectorizer.transform(X_df_test)

In [54]:
lr_model_tt = LogisticRegression(penalty="none", 
                              multi_class="multinomial",
                              solver="lbfgs").fit(X_train_2, y_train_2)
y_pred_lr_test_2 = lr_model_tt.predict(X_test_2)

CPU times: user 122 ms, sys: 12.4 ms, total: 134 ms
Wall time: 206 ms


In [57]:
print(classification_report(y_test_2, y_pred_lr_test_2))
print("ACCURACY: ",accuracy_score(y_test_2, y_pred_lr_test_2))

              precision    recall  f1-score   support

       False       0.86      0.83      0.85       137
        True       0.86      0.89      0.88       162

    accuracy                           0.86       299
   macro avg       0.86      0.86      0.86       299
weighted avg       0.86      0.86      0.86       299

ACCURACY:  0.862876254180602


In [111]:
df["label"] = df["label"].astype(int)
df.head()

,text,label
0,Putin After Announcing #CovidVaccine #Russian ...,1
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,1
2,4 of the vaccines Jared bought are expected to...,1
3,One day you will realize CDC Guidelines magica...,0
4,Im far from lying. Current CDC guidelines is ...,1


In [116]:
df_train, df_test = train_test_split(df)
df_train.shape

(896, 2)

In [117]:
df_test.shape

(299, 2)

In [172]:
# feature engineering
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(max_features=1000)
X_counts = count_vect.fit_transform(df.text)

# Transform counts to tf*idf vector
tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
X_tf = tf_transformer.transform(X_counts)



In [173]:
df_train, df_test = train_test_split(df)
X_train, X_test, y_train, y_test = train_test_split(X_tf, df.label)

In [174]:
def feature_function(text):
    return int("vaccin" in text)
train_new_feature_array = df_train.text.apply(feature_function)
test_new_feature_array = df_test.text.apply(feature_function)


In [154]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
# Add new feature to the document representation
X_train_final = np.insert(X_train.todense(), X_train.shape[1], train_new_feature_array, axis=1)
X_test_final = np.insert(X_test.todense(), X_test.shape[1], test_new_feature_array, axis=1)
# Train a text categorization model
clf = RandomForestClassifier()
clf.fit(X_train_final, df_train.label)


/Users/BharathBandaru/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


RandomForestClassifier()

In [155]:

pred = clf.predict(X_test_final)

/Users/BharathBandaru/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [156]:
accuracy_score(df_test.label,pred)

0.7090301003344481

In [164]:
lr2_model_fea = LogisticRegression(penalty="l2", 
                               solver="lbfgs",
                               multi_class="multinomial",
                               max_iter=1001,
                               C=10).fit(X_train_final, df_train.label)

/Users/BharathBandaru/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [165]:
y_lr2_test_fea = lr2_model_fea.predict(X_test_final)
print("ACCURACY: ",accuracy_score(df_test.label, y_lr2_test_fea))

ACCURACY:  0.6889632107023411


/Users/BharathBandaru/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
